Initialize the embedding function and the ChromaDB client

In [24]:
from chromadb.utils import embedding_functions
import chromadb

huggingface_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

chroma_client = chromadb.Client()
# client = chromadb.PersistentClient(path="/path/to/save/to")
chroma_client.delete_collection(name="knowledge_collection")

knowledge_collection = chroma_client.create_collection(name="knowledge_collection", embedding_function=huggingface_ef)

Take data from a pdf file

In [25]:
import fitz
doc = fitz.open("/Users/lucacordioli/Documents/Lavori/polimi/TESI/visionHelperSrv/data/test.pdf")

pdf_texts = ""

for page in doc: # iterate the document pages
    pdf_texts += page.get_text()
    

Add data to chorma db

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=300, chunk_overlap=50)

# clear the collection
for i, page in enumerate(doc):
    page_text = page.get_text()
    doc_splits = text_splitter.split_text(page_text)
    knowledge_collection.add(
        documents=doc_splits,
        metadatas=[{'page': i+1} for _ in doc_splits],
        ids=[f"id_{i}_{j}" for j in range(len(doc_splits))]
    )

Take data from web page

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
urls = [
    "https://www.apple.com/newsroom/2024/05/apple-reports-second-quarter-results/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

Add data to chorma db

In [ ]:
doc_splits = text_splitter.split_documents(docs_list)

knowledge_collection.add(
    documents=[doc.page_content for doc in doc_splits],
    metadatas=[doc.metadata for doc in doc_splits],
    ids=[f"id_{doc}" for doc in range(len(doc_splits))]
)

# From question to answer

Query to ChromaDB

In [ ]:
question = "What is Luca educational path?"
system_message = ""

results = knowledge_collection.query(
    query_texts=[question],
    n_results=3,
    include=["documents"]
)

context = ""
for doc in results["documents"]:
    context += "\n\n".join(doc)
    
print(context)

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOllama(model="llama3:8b", temperature=0, max_tokens=2048,
                 stop=["<|start_header_id|>", "<|end_header_id|>", "<|eot_id|>", "<|reserved_special_token"])

prompt = PromptTemplate(
    template="""<|begin_of_text|>
      <|start_header_id|>system<|end_header_id|>You are a question-answer task assistant. Use the following portion of the context to answer the question. If you do not know the answer, simply answer that you do not know the answer. Use a maximum of 3 sentences to answer concisely.<|eot_id|>
      <|start_header_id|>user<|end_header_id|>
      Question: {question}
      Context: {context}
      Answer: <|eot_id|>
      <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["system_message", "question", "context"],
)

chain = prompt | llm | StrOutputParser()

for chunk in chain.stream({"question": question, "system_message": system_message, "context": context}):
    print(chunk, end="", flush=True)

# From piece id to pdf page

Load json file

In [27]:
import json

index = 3

with open('/Users/lucacordioli/Documents/Lavori/polimi/TESI/visionHelperSrv/data/data.json', 'r') as file:
    data = json.load(file)
    
    obj = data['medias'][index]
    
    item_id = obj['id']
    name = obj['name']
    caption = obj['caption']
    guid = obj['guid']
    media_type = obj['mediaType']['value']

    query_text = f"ID: {item_id}, Name: {name}, Caption: {caption}, GUID: {guid}, MediaType: {media_type}"
    
    print(query_text)


ID: 13483, Name: Hand, Caption: Hand, GUID: 7b6d3c28-5650-49aa-9c6a-aeec1cf5524a, MediaType: 3D Object


Query to ChromaDB

In [ ]:
results = knowledge_collection.query(
    query_texts=[query_text],
    n_results=3,
    include=["documents", "metadatas"]
)

print("Page: " + str(results["metadatas"][0][0]["page"]))
print("Document: " + results["documents"][0][0])

# From request to piece id

Embed json in chroma db

In [28]:
pieces_collection = chroma_client.create_collection(name="pieces", embedding_function=huggingface_ef)
for obj in data['medias']:
    pieces_collection.add(
        documents=[str(obj)],
        metadatas=[{"source": "pezzo_meccanico.json"}],
        ids=[f"id_{obj['id']}"]
    )


UniqueConstraintError: Collection pieces already exists

Query knowledge collection in chromadb

In [29]:
question = "Where is the ForeArm?"
system_message = ""

results = knowledge_collection.query(
    query_texts=[question],
    n_results=3,
    include=["documents"]
)

context = ""
for doc in results["documents"]:
    context += "\n\n".join(doc)
    
    
prompt_embedding = question +  "\n\n" + context


In [32]:
results = pieces_collection.query(
    query_texts=[prompt_embedding],
    n_results=3,
    include=["documents", "metadatas"]
)
print(results)

{'ids': [['id_13482', 'id_13483', 'id_13485']], 'distances': None, 'metadatas': [[{'source': 'pezzo_meccanico.json'}, {'source': 'pezzo_meccanico.json'}, {'source': 'pezzo_meccanico.json'}]], 'embeddings': None, 'documents': [["{'id': 13482, 'name': 'ForeArm', 'caption': 'ForeArm', 'guid': 'dc41c874-782a-4499-a828-2fdf3f996855', 'extension': 'glb', 'thumbnailExtension': 'png', 'fileSize': 2215632, 'contentType': 'application/octet-stream', 'mediaType': {'key': 4, 'value': '3D Object'}, 'creationDate': '2024-04-15T09:41:04.2957045', 'hasThumbnail': True, 'parentMediaId': None, 'carryOverId': 13482, 'steps': ['1713172892694'], 'children': [], 'customParameters': None}", "{'id': 13483, 'name': 'Hand', 'caption': 'Hand', 'guid': '7b6d3c28-5650-49aa-9c6a-aeec1cf5524a', 'extension': 'glb', 'thumbnailExtension': 'png', 'fileSize': 1185264, 'contentType': 'application/octet-stream', 'mediaType': {'key': 4, 'value': '3D Object'}, 'creationDate': '2024-04-15T09:41:04.2957045', 'hasThumbnail': Tr